# Telemetry Collection Agent (TCA) loads via COA

In [ ]:
%connect Transcend

In [ ]:
select * from dbc.dbcinfo

<br><br><br>
## Analysis: how much data per day? 
Assume 3x compression

In [127]:
Select cds.DataBaseName, cds.TableName, 
       sum(CurrentPerm)/1e6 as CurrPerm_MB, max(DateCount) as Date_Cnt,
       (CurrPerm_MB / Date_Cnt * 3.00)(Decimal(18,2)) as MB_Per_Day
from dbc.TableSizeV as cds
join   (select 'PDCRData' as DatabaseName, 'ResUsageSPDSK_Hst' as TableName, 
        count(distinct TheDate) as DateCount from PDCRInfo.ResUsageSPDSK_Hst
        union all
        select 'PDCRData' as DatabaseName, 'ResUsageSVPR_Hst' as TableName, 
        count(distinct TheDate) as DateCount from PDCRInfo.ResUsageSVPR_Hst
        union all
        select 'PDCRData' as DatabaseName, 'ResUsageSPMA_Hst' as TableName, 
        count(distinct TheDate) as DateCount from PDCRInfo.ResUsageSPMA_Hst
        union all
        select 'DBC' as DatabaseName, 'ResUsageSVPR' as TableName, 
        count(distinct TheDate) as DateCount from DBC.ResUsageSVPR
        union all
        select 'DBC' as DatabaseName, 'ResUsageSPMA' as TableName, 
        count(distinct TheDate) as DateCount from DBC.ResUsageSPMA
        union all
        select 'DBC' as DatabaseName, 'ResUsageSPDSK' as TableName, 
        count(distinct TheDate) as DateCount from DBC.ResUsageSPDSK) as cnt
 on trim(cds.DatabaseName) = trim(cnt.DatabaseName)
and trim(cds.TableName) = trim(cnt.TableName)
where cds.DatabaseName in ('PDCRData','DBC')
  and cds.TableName like ANY ('ResUsageSVPR%','ResUsageSPMA%','ResUsageSPDSK%')
Group by 1,2  Order by 1,2

DataBaseName,TableName,CurrPerm_MB,Date_Cnt,MB_Per_Day
DBC,ResUsageSpdsk,16237.867008,96,507.43
DBC,ResUsageSpma,667.578368,96,20.86
DBC,ResUsageSvpr,52170.502144,96,1630.33
PDCRDATA,ResUsageSpdsk_Hst,2592.137216,94,82.73
PDCRDATA,ResUsageSpma_Hst,1456.193536,1461,2.99
PDCRDATA,ResUsageSvpr_Hst,8720.314368,94,278.31


<br><br><br>
# Build supporting datalab objects 
- Stage tables  (global temp)
- Target Tables (for demo purposes only, will eventually be TCA)
- Stored Procs  (move data from stage to target, for demo purposes only)

<br><br><br>
## ResUsage STAGE -- Build GTT Tables

In [ ]:
drop table adlste_coa_stg."stg_tca_dbcinfo"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_dbcinfo" as
(select cast('' as varchar(128)) as Site_ID, a.* from dbc.dbcinfo as a) 
with no data
no primary index 
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagesvpr_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagesvpr_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from PDCRInfo.ResUsageSVPR_hst as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagesvpr_dbc_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagesvpr_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from dbc.ResUsageSVPR as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagespma_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagespma_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from PDCRInfo.ResUsageSPMA_hst as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagespma_dbc_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagespma_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from dbc.ResUsageSPMA as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagespdsk_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagespdsk_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from PDCRInfo.ResUsageSPDSK_hst as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa_stg."stg_tca_resusagespdsk_dbc_16.20" 

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."stg_tca_resusagespdsk_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.* from dbc.ResUsageSPDSK as a) 
with no data
no primary index
ON COMMIT PRESERVE ROWS;

<br><br><br>
## ResUsage TARGET -- Normal Tables
this is for demo purposes only... eventually this will reside in another database for TCA upload

### dbcinfo

In [ ]:
drop table adlste_coa."coat_tca_dbcinfo"

In [ ]:
CREATE TABLE adlste_coa."coat_tca_dbcinfo" as
(select cast('' as varchar(128)) as Site_ID, cast(DATE as DATE format'yyyy-mm-dd') as LogDate,
        a.*, cast(0 as INT) as Run_ID from dbc.dbcinfo as a) 
with no data
primary index (Site_ID);

In [ ]:
replace view adlste_coa."coa_tca_dbcinfo" as locking row for access
Select * from adlste_coa."coat_tca_dbcinfo";

### resusageSVPR 16.20 PDCR

In [ ]:
drop table adlste_coa."coat_tca_resusagesvpr_pdcr_16.20" 

In [ ]:
CREATE TABLE adlste_coa."coat_tca_resusagesvpr_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from PDCRInfo.ResUsageSVPR_hst as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa."coa_tca_resusagesvpr_pdcr_16.20" as locking row for access
select * from adlste_coa."coat_tca_resusagesvpr_pdcr_16.20" 

### resusageSVPR 16.20 DBC

In [ ]:
drop table adlste_coa."coat_tca_resusagesvpr_dbc_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa."coat_tca_resusagesvpr_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from dbc.ResUsageSVPR as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view  adlste_coa."coa_tca_resusagesvpr_dbc_16.20" as locking row for access
select * from  adlste_coa."coat_tca_resusagesvpr_dbc_16.20"

### resusageSPMA 16.20 PDCR

In [ ]:
drop table adlste_coa."coat_tca_resusagespma_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa."coat_tca_resusagespma_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from PDCRInfo.ResUsageSPMA_hst as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa."coa_tca_resusagespma_pdcr_16.20" as locking row for access
select * from adlste_coa."coat_tca_resusagespma_pdcr_16.20"

### resusageSPMA 16.20 DBC

In [ ]:
drop table adlste_coa_stg."coat_tca_resusagespma_dbc_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg."coat_tca_resusagespma_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from dbc.ResUsageSPMA as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa_stg."coa_tca_resusagespma_dbc_16.20" as locking row for access
select * from adlste_coa_stg."coat_tca_resusagespma_dbc_16.20"

### resusageSPDSK 16.20 PDCR

In [ ]:
drop table adlste_coa."coat_tca_resusagespdsk_pdcr_16.20"

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa."coat_tca_resusagespdsk_pdcr_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from PDCRInfo.ResUsageSPDSK_hst as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa."coa_tca_resusagespdsk_pdcr_16.20" as locking row for access
select * from adlste_coa."coat_tca_resusagespdsk_pdcr_16.20"

### resusageSPDSK 16.20 DBC

In [ ]:
drop table adlste_coa."coat_tca_resusagespdsk_dbc_16.20" 

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa."coat_tca_resusagespdsk_dbc_16.20" as
(select cast('' as varchar(128)) as Site_ID, a.*, cast(0 as INT) as Run_ID from dbc.ResUsageSPDSK as a) 
with no data
primary index (Site_ID, TheDate, TheTime);

In [ ]:
replace view adlste_coa."coa_tca_resusagespdsk_dbc_16.20" as locking row for access
select * from adlste_coa."coat_tca_resusagespdsk_dbc_16.20" 

<br><br><br>
## Stored Proc to merge DBC.DBCInfo 
This is for demo purposes... longer term this SP may live elsewhere, or insert into a different target location. <br>
Also note, this SP is templated for COA audit processes, hence the extra logging.

### DBCInfo

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_tca_dbcinfo () 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    DECLARE spversion VARCHAR(128);
    DECLARE runid  INTEGER;
    DECLARE runtype VARCHAR(128);
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    DECLARE siteid VARCHAR(128);
    DECLARE procname VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128);
    DECLARE rowcount bigint;

    SET procname = 'tca_dbcinfo';
    SET callingsp = 'adlste_coa.sp_' || procname;
    SET tablename = 'adlste_coa.coat_' || procname;
    -------------
    SET spversion = 'v1';
    SET runid = 0;
    SET siteid = 'All';
    SET runtype = 'COA Run, TCA';

    SET startdate = DATE;
    SET enddate = DATE;

    /* collect startdate, enddate, and siteid from Stage */
    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(LogDate)(DATE) as StartDate
    ,max(LogDate)(DATE) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_tca_dbcinfo
    Group by 1
    ;
    
    CALL adlste_coa.sp_get_rowcount(:tablename, rowcount);
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, :runtype,
        'START', 'record count: ' || cast(rowcount as varchar(32)) ) ;   /*  run_status, run_msg */
    /* ------------------------
       end setup, begin sp logic
       ------------------------
    */
 
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, :runtype, 
        'STEP 01', 'Delete Duplicates from Target') ;   
        DELETE FROM adlste_coa.coat_tca_dbcinfo 
        WHERE (Site_ID, LogDate) IN
        (Select Site_ID, LogDate From adlste_coa_stg.stg_tca_dbcinfo );

    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'STEP 02', 'Insert into Target') ;   
        Insert into adlste_coa.coat_tca_dbcinfo 
        Select a.*, :runid
        from adlste_coa_stg.stg_tca_dbcinfo ; 
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'STEP 03', 'Delete from GTT') ;   
        delete from adlste_coa_stg.stg_tca_dbcinfo;
                        
            
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
            'STEP 04', 'Collect Stats') ;   
        collect stats column(Site_ID) on adlste_coa.coat_tca_dbcinfo;
            
    /* ------------------------
       end sp logic, begin clean-up
       ------------------------
    */   
    CALL adlste_coa.sp_get_rowcount(:tablename, rowcount);
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'END', 'record count: ' || cast(rowcount as varchar(32)) ) ;

END;


In [ ]:
REPLACE PROCEDURE adlste_coa.sp_tca_resusage(suffix VARCHAR(16), collection VARCHAR(16), dbsversion VARCHAR(16)) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    DECLARE spversion VARCHAR(128);
    DECLARE runid  INTEGER;
    DECLARE runtype VARCHAR(128);
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    DECLARE siteid VARCHAR(128);
    DECLARE procname VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE stagename VARCHAR(128);
    DECLARE callingsp VARCHAR(128);
    DECLARE rowcount bigint;
    DECLARE sqltxt VARCHAR(16000);

    SET procname = 'tca_resusage' || trim(:suffix) ||'_'|| trim(:collection) ||'_'|| trim(:dbsversion);
    SET callingsp = 'adlste_coa.sp_tca_resusage';
    SET tablename = 'adlste_coa.coat_' || procname;    -- "coat_tca_resusagesvpr_dbc_16.20"
    SET stagename = 'adlste_coa_stg.stg_' || procname; -- "stg_tca_resusagespma_pdcr_16.20"
    -------------
    SET spversion = 'v1';
    SET runid = 0;
    SET siteid = 'All';
    SET runtype = 'COA Run, TCA';

    SET startdate = DATE;
    SET enddate = DATE;

    /* collect startdate, enddate, and siteid from Stage */
    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(TheDate)(DATE) as StartDate
    ,max(TheDate)(DATE) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_tca_dbcinfo
    Group by 1
    ;
    
    CALL adlste_coa.sp_get_rowcount(:tablename, rowcount);
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, :runtype,
        'START', 'record count: ' || cast(rowcount as varchar(32)) ) ;   /*  run_status, run_msg */
    /* ------------------------
       end setup, begin sp logic
       ------------------------
    */
 
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, :runtype, 
        'STEP 01', 'Delete Duplicates from Target') ;   
        SET sqltxt = ' DELETE FROM ' || :tablename || 
                     ' WHERE (Site_ID, TheDate) IN (Select Site_ID, TheDate From '|| :stagename || ' )' ;

    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'STEP 02', 'Insert into Target') ;   
        SET sqltxt = ' INSERT into ' || :tablename || ' Select a.*, ' || :runid || ' FROM ' || :stagename ;
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'STEP 03', 'Delete from GTT') ;   
        SET sqltxt = ' DELETE FROM ' || :stagename ;
                        
            
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
            'STEP 04', 'Collect Stats') ;   
        set sqltxt = 'collect stats column(Site_ID) on ' || :tablename ;
            
    /* ------------------------
       end sp logic, begin clean-up
       ------------------------
    */   
    CALL adlste_coa.sp_get_rowcount(:tablename, rowcount);
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'END', 'record count: ' || cast(rowcount as varchar(32)) ) ;

END;
